# Laura Souza de Carvalho - RM: 556320
# Ali Andrea Mamani Molle - RM: 558052
# Lucas Vasquez - RM: 555159
# Guilherme Linard - RM: 555768
# Lucas Queiroz - RM: 556323

# Otimização de Estoque com Programação Dinâmica

# Problema
Nas unidades de diagnóstico, o consumo diário de insumos (reagentes e descartáveis) não é registrado com precisão, dificultando o controle de estoque e a previsão de reposição. Diante disso, é necessário modelar este problema usando Programação Dinâmica para propor uma solução que melhore a visibilidade do consumo e reduza desperdícios.

Baseamos nosso modelo no problema de Dimensionamento de Lote (Lot-Sizing).Estado ($i$):$C(i)$ = O custo mínimo para satisfazer toda a demanda a partir do dia $i$ até o final, assumindo que o estoque no dia $i$ é zero.Decisão ($j$):No dia $i$ (com estoque zero), decidimos fazer um pedido para cobrir a demanda de todos os dias desde $i$ até $j$ (onde $i \le j < N$).Função Objetivo:Encontrar $C(0)$, que é o custo mínimo total para todo o período (do dia 0 até $N-1$).Função de Transição (Recorrência):O custo $C(i)$ é o mínimo entre todas as decisões $j$ possíveis. O custo de uma decisão $j$ é:(Custo Fixo $K$) + (Custo de Estoque para $D_i...D_j$) + (Custo do subproblema $C(j+1)$).$$C(i) = \min_{i \le j < N} \left\{ K + \sum_{k=i}^{j} h \cdot D_k \cdot (k-i) + C(j+1) \right\}$$Caso Base:$C(N) = 0$. O custo para atender a demanda após o último dia é zero.

Tipo de Célula: Texto (Markdown)
Este bloco importa sys (para recursão) e define os dados do problema:

D: Lista de demandas diárias.

N: Total de dias.

K: Custo fixo por pedido.

h: Custo de manter 1 unidade em estoque por 1 dia.

In [1]:
import sys

# Aumenta o limite de recursão para a versão recursiva pura
sys.setrecursionlimit(2000)

# --- Dados do Problema ---
# D = Demandas (consumo) para os próximos N dias
D = [40, 10, 10, 100, 30] 
N = len(D)

K = 100  # Custo fixo de fazer um pedido (ex: R$ 100)
h = 2    # Custo de manter 1 unidade em estoque por 1 dia (ex: R$ 2)

print(f"Dados de Entrada:")
print(f"Horizonte de {N} dias.")
print(f"Demandas (D): {D}")
print(f"Custo de Pedido (K): R$ {K}")
print(f"Custo de Holding (h): R$ {h} por unidade/dia")

Dados de Entrada:
Horizonte de 5 dias.
Demandas (D): [40, 10, 10, 100, 30]
Custo de Pedido (K): R$ 100
Custo de Holding (h): R$ 2 por unidade/dia


Versão 1: Recursiva Pura (Top-Down)

Esta função implementa a fórmula de recorrência diretamente. Para cada dia i, ela testa todas as decisões j (pedir até o dia j) e chama a si mesma para o dia j+1. É lenta porque recalcula os mesmos dias várias vezes.

In [2]:
def resolver_recursivo(i, N, D, K, h):
    """
    Calcula o custo mínimo a partir do dia 'i' usando recursão pura.
    """
    # Caso Base: Se estamos no dia N (ou além), não há mais demanda/custo.
    if i == N:
        return 0

    min_custo_total = float('inf')

    # Decisão: Testar todas as possíveis datas 'j' para o fim do lote
    for j in range(i, N):
        # 1. Custo Fixo do Pedido
        custo_pedido = K

        # 2. Custo de Manutenção de Estoque
        custo_holding = 0
        for k in range(i, j + 1):
            # Demanda D[k] é mantida por (k-i) dias
            custo_holding += h * D[k] * (k - i)

        # 3. Custo do Subproblema Restante (a partir do dia j+1)
        custo_subproblema = resolver_recursivo(j + 1, N, D, K, h)

        # Custo total para esta decisão 'j'
        custo_total_decisao = custo_pedido + custo_holding + custo_subproblema

        # Queremos o mínimo entre todas as decisões 'j'
        min_custo_total = min(min_custo_total, custo_total_decisao)

    return min_custo_total

Versão 2: Memorização (Top-Down com Memo)

Esta é a versão recursiva otimizada. Ela usa um dicionário memo para guardar resultados. Antes de calcular o custo para um dia i, ela verifica se ele já está no memo. Se sim, retorna o valor salvo. Se não, calcula e salva no memo antes de retornar.

In [3]:
def resolver_memoization(i, N, D, K, h, memo):
    """
    Calcula o custo mínimo a partir do dia 'i' usando recursão com memorização.
    'memo' é um dicionário para guardar resultados já calculados.
    """
    # Caso Base
    if i == N:
        return 0

    # Verifica se o resultado para o estado 'i' já foi calculado
    if i in memo:
        return memo[i]

    min_custo_total = float('inf')

    # A lógica da decisão é idêntica à recursiva
    for j in range(i, N):
        custo_pedido = K
        custo_holding = 0
        for k in range(i, j + 1):
            custo_holding += h * D[k] * (k - i)

        # A única mudança é passar o 'memo' na chamada recursiva
        custo_subproblema = resolver_memoization(j + 1, N, D, K, h, memo)

        custo_total_decisao = custo_pedido + custo_holding + custo_subproblema
        min_custo_total = min(min_custo_total, custo_total_decisao)

    # Armazena o resultado no 'memo' antes de retornar
    memo[i] = min_custo_total
    return min_custo_total

Versão 3: Iterativa (Bottom-Up)

Esta versão não usa recursão. Ela cria uma tabela (array) dp e a preenche "de trás para frente" (do último dia até o dia 0). Para calcular o custo dp[i], ela usa os valores dp[j+1] que já foram calculados e estão na tabela. Esta é a abordagem mais eficiente.

In [4]:
def resolver_bottom_up(N, D, K, h):
    """
    Calcula o custo mínimo de forma iterativa, preenchendo a tabela dp.
    """
    # dp[i] = Custo mínimo para satisfazer a demanda do dia 'i' até N-1.
    # Tamanho N+1 para incluir o caso base dp[N].
    dp = [0] * (N + 1)

    # Caso Base: dp[N] = 0 (custo após o último dia é 0)

    # Preenchemos a tabela "de trás para frente"
    # Começamos do último dia (N-1) e vamos até o dia 0
    for i in range(N - 1, -1, -1):
        
        min_custo_para_i = float('inf')
        
        # Testamos todas as decisões 'j' (de 'i' até N-1)
        for j in range(i, N):
            custo_pedido = K
            custo_holding = 0
            for k in range(i, j + 1):
                custo_holding += h * D[k] * (k - i)

            # Custo do subproblema é pego diretamente da tabela 'dp'
            # Esta é a resposta para C(j+1)
            custo_subproblema = dp[j + 1]

            custo_total_decisao = custo_pedido + custo_holding + custo_subproblema
            min_custo_para_i = min(min_custo_para_i, custo_total_decisao)
            
        # Armazenamos o custo mínimo para o estado 'i'
        dp[i] = min_custo_para_i

    # A resposta final é o custo mínimo começando do dia 0
    return dp[0]

Este bloco final executa as três funções (Recursiva, Memorização e Bottom-Up) com os mesmos dados. O assert no final verifica se todas elas produziram exatamente o mesmo resultado, garantindo que nossa lógica está correta.

In [5]:
# ---
# Execução e Verificação (Garantia de Resultados)
# ---

print(f"--- Iniciando Cálculo ---")

# 1. Resolvendo com Recursão Pura
custo_rec = resolver_recursivo(0, N, D, K, h)
print(f"[Recursivo Puro] Custo Mínimo Total: R$ {custo_rec}")

# 2. Resolvendo com Memorização
memo = {} # Zera o 'memo'
custo_memo = resolver_memoization(0, N, D, K, h, memo)
print(f"[Memorização]    Custo Mínimo Total: R$ {custo_memo}")

# 3. Resolvendo com Bottom-Up
custo_bu = resolver_bottom_up(N, D, K, h)
print(f"[Bottom-Up]      Custo Mínimo Total: R$ {custo_bu}")

# 4. Verificação
assert custo_rec == custo_memo == custo_bu

print("\n--- VERIFICAÇÃO ---")
print("OK: Todos os métodos produziram o mesmo resultado.")

--- Iniciando Cálculo ---
[Recursivo Puro] Custo Mínimo Total: R$ 320
[Memorização]    Custo Mínimo Total: R$ 320
[Bottom-Up]      Custo Mínimo Total: R$ 320

--- VERIFICAÇÃO ---
OK: Todos os métodos produziram o mesmo resultado.
